> Proszę zaproponować algorytm, który mając dane dwa słowa $ A $ i $ B $ o długości $ n $, każde nad alfabetem długości $ k $, sprawdza, czy $ A $ i $ B $ są swoimi anagramami.

1. Proszę zaproponować rozwiązanie działające w czasie $ O(n + k) $.

2. Proszę zaproponować rozwiązanie działające w czasie $ O(n) $ (proszę zwrócić uwagę, że $ k $ może być dużo większe od $ n $ — np. dla alfabetu unicode; złożoność pamięciowa może być rzędu $ O(n + k) $).

###### Proszę zaimplementować oba algorytmy.

# 1. podpunkt
##### Algorytm o złożoności obliczeniowej $ O(n + k) $

### Omówienie algorytmu

Idea algorytmu jest taka, aby w tablicy długości $ k $, której indeksy odpowiadają numerom (pomniejszonym o wartość pierwszego znaku alfabetu - uznajemy, że alfabet jest tworzony przez kolejne znaki UNICODE, a więc np. pomiędzy każdymi dwiema literami danego alfabetu nie znajduje się żaden znak, który nie należy do tego alfabetu) UNICODE odpowiednich znaków alfabetu, przechowywać liczniki wartości. Ponieważ anagramy to słowa tej samej długości, zbudowane z tej samej liczby takich samych znaków, w których różni się jedynie kolejność tych znaków (jedno słowo powtało poprzez permutację liter drugiego słowa), wystarczy nam policzyć ile danych liter posiada każde słowo i odpowiednio zwiększać licznik, jeżeli pierwsze słowo zawiera daną literę oraz zmniejszać licznik, jeżeli drugie słowo posiada daną literę (można odwrotnie). Na koniec wystarczy przebiec liniowo po tablicy liczników długości $ k $ i sprawdzać, czy licznik na każdym z pól ma wartość $ 0 $ (tzn. oba słowa zawierają tyle samo znaków o danym kodzie). Jeżeli natrafimy na liczbę różną od $ 0 $, oznacza to, że słowa nie są anagramami, w przeciwnym razie, po zakończniu przebiegu bez przerwania pętli, słowa są anagramami.

Warto nadmienić, że złożoność pamięciowa w tym przypadku również wynosi $ O(n + k) $.

### Implementacja algorytmu

In [1]:
def are_anagrams(s1, s2):
    # Words of different lengths cannot be anagrams
    if len(s1) != len(s2): return False
    # Find the maximum and the minimum value of the UNICODE code
    # of the letters in both strings and create an array of to store counters
    # (It's essential to know the code of the letter from boths strings which
    # appears in an alphabet first in order to access the indices of counts
    # array properly (as we are not given a char code of the first letter of an
    # alphabet)) (maximum value is not required as we know how many letters
    # there are in an alphabet but this ensures that we won't create redundant
    # array slots).
    s1_min_char, s1_max_char = minmax(s1)
    s2_min_char, s2_max_char = minmax(s2)
    min_char_ord = min(ord(s1_min_char), ord(s2_min_char))
    max_char_ord = min(ord(s1_max_char), ord(s2_max_char))
    # Allocate memory for counters
    counters = [0] * (max_char_ord - min_char_ord + 1)
    
    # Loop over both strings and increment or decrement appropriate counters
    for char in s1: counters[ord(char) - min_char_ord] += 1
    for char in s2: counters[ord(char) - min_char_ord] -= 1
        
    # Loop over a counters array and check if all counters are set to 0
    for counter in counters:
        # If is non-zero, return False as words cannot be anagrams
        if counter: return False
    # If a loop above has finished, return True
    return True
    
    
def minmax(iterable):
    # We assume that an iterable supports indexing (to handle sets or dictionaries
    # iteration we can use an iterator object but in this example this is overkill).
    # Store the last value in order to ease odd-length iterable values comparisons.
    max_val = min_val = iterable[-1]  
    
    for i in range(1, len(iterable), 2):
        if iterable[i] > iterable[i-1]:
            if iterable[i] > max_val:   max_val = iterable[i]
            if iterable[i-1] < min_val: min_val = iterable[i-1]
        else:
            if iterable[i] < min_val:   min_val = iterable[i]
            if iterable[i-1] > max_val: max_val = iterable[i-1]
                
    # Return the minimum and the maximum value
    return min_val, max_val

###### Kilka testów

In [2]:
print(are_anagrams('aaaa', 'aaaa'))
print(are_anagrams('aaaa', 'aaaaa'))
print(are_anagrams('😂', '😂'))
print(are_anagrams('😂🤣👀🎂✨🎁', '🎂🎁😂👀✨🤣'))
print(are_anagrams('kot', 'kto'))
print(are_anagrams('ola ala ula ela', 'ela ala ola ula'))
print(are_anagrams('ola ala ula ela', 'ela alaola ula'))

True
False
True
True
True
True
False


# 2. podpunkt
##### Algorytm o złożoności obliczeniowej $ O(n) $ i pamięciowej maksymalnie $ O(n + k) $

## I Sposób

### Omówienie algorytmu

W tej implementacji zakładamy, że zaalokowanie tablicy długości $ k $ zajmuje czas $ O(1) $, jak ma to miejsce w językach, w których tablice nie są dynamiczne, np. w językach z rodziny C. Wówczas można zauważyć, że nie musimy przebiegać po załej $ k $-elementowej tablicy liczników, a jedynie wystarczy podczas aktualizowania liczników dla drugiego wyrazu, sprawdzać za każdym razem, czy licznik nie zszedł poniżej zera. Jeżeli tak, możemy zakończyć aktualizowanie liczników i zwrócić fałsz, bo już "nie nadrobimy" wartości (nie da się jej przywrócić do zera, bo podczas aktualizacji liczników dla drugiego wyrazu, dekrementujemy liczniki).

###### UWAGA:

Warto zauważyć, że jeżeli w drugim z wyrazów będzie mniej wystąpień danego znaku niż w pierwszym wyrazie, sprawdzany przez nas warunek na zejście licznika poniżej 0 nie przerwie pętli (nie wykryjemy tego, że dana litera występuje w pierwszym wyrazie więcej razy niż w drugim). Jednakże NIE JEST TO PROBLEM, bo sprawdzamy najpierw, czy oba wyrazy mają tę samą długość. Jesteśmy więc pewni, że skoro w drugim wyrazie jest mniej wystąpień danej litery, inna litera musi wystąpić więcej razy niż w pierwszym wyrazie (tzn. nigdy nie zajdzie sytuacja, że dwa wyrazy są równej długości, ale w drugim wyrazie liczba wystąpień wszystkich liter jest mniejsza niż liczba wystąpień liter w pierwszym wyrazie). Wynika to z faktu, że nawet jeśli każde z liter wyrazu pierwszego występuje w wyrazie drugim mniej razy niż w wyrazie pierwszym, to w drugim wyrazie znajdzie się zawsze taka litera, która w pierwszym wyrazie nie występuje. Wtedy licznik dla danej litery zostanie zmniejszony z 0 do -1 i funkcja zwróci wartość fałsz.

### Implementacja algorytmu

In [3]:
def are_anagrams(s1, s2):
    # Words of different lengths cannot be anagrams
    if len(s1) != len(s2): return False
    # Find the maximum and the minimum value of the UNICODE code
    # of the letters in both strings and create an array of to store counters
    # (It's essential to know the code of the letter from boths strings which
    # appears in an alphabet first in order to access the indices of counts
    # array properly (as we are not given a char code of the first letter of an
    # alphabet)) (maximum value is not required as we know how many letters
    # there are in an alphabet but this ensures that we won't create redundant
    # array slots).
    s1_min_char, s1_max_char = minmax(s1)
    s2_min_char, s2_max_char = minmax(s2)
    min_char_ord = min(ord(s1_min_char), ord(s2_min_char))
    max_char_ord = min(ord(s1_max_char), ord(s2_max_char))
    # Allocate memory for counters
    counters = [0] * (max_char_ord - min_char_ord + 1)
    
    # Loop over both strings and increment or decrement appropriate counters
    for char in s1: 
        counters[ord(char) - min_char_ord] += 1
        
    for char in s2: 
        idx = ord(char) - min_char_ord
        counters[idx] -= 1
        # If we decreased a counter too much (the second word thet means the second
        # word has more letters of this code so these words cannot be anagrams).
        if counters[idx] < 0:
            return False
        
    return True
    
    
def minmax(iterable):
    # We assume that an iterable supports indexing (to handle sets or dictionaries
    # iteration we can use an iterator object but in this example this is overkill).
    # Store the last value in order to ease odd-length iterable values comparisons.
    max_val = min_val = iterable[-1]  
    
    for i in range(1, len(iterable), 2):
        if iterable[i] > iterable[i-1]:
            if iterable[i] > max_val:   max_val = iterable[i]
            if iterable[i-1] < min_val: min_val = iterable[i-1]
        else:
            if iterable[i] < min_val:   min_val = iterable[i]
            if iterable[i-1] > max_val: max_val = iterable[i-1]
                
    # Return the minimum and the maximum value
    return min_val, max_val

###### Kilka testów

In [4]:
print(are_anagrams('aaaa', 'aaaa'))
print(are_anagrams('aaaa', 'aaaaa'))
print(are_anagrams('😂', '😂'))
print(are_anagrams('😂🤣👀🎂✨🎁', '🎂🎁😂👀✨🤣'))
print(are_anagrams('kot', 'kto'))
print(are_anagrams('ola ala ula ela', 'ela ala ola ula'))
print(are_anagrams('ola ala ula ela', 'ela alaola ula'))

True
False
True
True
True
True
False


## II Sposób

### Omówienie algorytmu

W tym sposobie zakładamy, że tablica na kolejne znaki alfabetu jest już odgórnie zaalokowana i akceptuje wszystkie znaki alfabetu UNICODE. Ponieważ nie mamamy danego kodu pierwszego znaku alfabetu, alokujemy od razu globalną tablicę na wszystkie możliwe przypadki. Ten algorytm jest najgorszy, jeżeli chodzi o złożoność pamięciową, ale mamy powiedziane w 2. podpunkcie, że pamięć nie ma znaczenia (fragment: "$ k $ może być dużo większe od $ n $ — np. dla alfabetu unicode" oraz fragment: "złożoność pamięciowa może być rzędu $ O(n + k) $"). Czyli złożoność pamięciowa nas nie obchodzi. Jeżeli chodzi o złożoność obliczeniową, to jest ona tu najlepsza, ponieważ nie musimy szukać znaku o największym oraz znaku u najmniejszym kodzie UNICODE, jak miało to miejsce wyżej (mamy około $ \frac{3}{2}n \cdot 2 \cdot c = 3nc $ mniej porównań, gdzie $ c $ - pewna stała, bo funkcja $ minmax $ działa ze złożonością $ O(\frac{3}{2}n) = O(n) $). Musimy zatem pamiętać o wyzerowaniu liczników najpierw, co zajmie nam ok. $ O(2n \cdot c) = O(n) $ operacji (wystarczy nam zerowanie dla jednego wyrazu). Wynika to z faktu, iż tablica jest odgórnie zaalokowana i mogą w niej być pozostałości po poprzednich porównaniach wyrazów (oraz w innych językach programowania zazwyczaj tworzenie tablicy odbywa się jedynie jako utworzenie wskaźnika do pierwszego elementu bez inicjalizacji kolejnych slotów, więc występują na nich losowe wartości).

### Implementacja algorytmu

##### Implementacja klasy pomocniczej, która "udaje" alokowanie tablicy, działające podobnie do języków z rodziny C.

In [5]:
import random

def rand_int(bits=16):
    return random.randint(0, 2 ** bits)

class Array:
    def __init__(self, size):
        self.values = [rand_int() for _ in range(size)]
        
    def __getitem__(self, idx):
        if 0 <= idx < len(self.values):
            return self.values[idx]
        # Imitate access to the index beyond of an the array
        return rand_int()
    
    def __setitem__(self, idx, val):
        if 0 <= idx < len(self.values):
            self.values[idx] = val

##### Właściwy algorytm

In [6]:
def are_acronyms_init():
    # Allocate a memory for the whole UNICODE alphabet (only once)
    # as nonlocal variable accessible by the inner function only.
    counts = Array(2 ** 16)
    
    # Return a function which will be used to compare strings
    def are_acronyms(s1, s2):
        if len(s1) != len(s2): return False
        # Fill the necessary counters with zeros (we don't have
        # to do this for boths strings)
        for char in s1: counts[ord(char)] = 0
            
        # Increment counters for characters in the first string
        for char in s1: counts[ord(char)] += 1
        # Decrement counters for characters in the second string
        for char in s2: counts[ord(char)] -= 1
        
        # Check if all counters of the first string characters are
        # equal to 0. Note that we don't have to check counters for
        # the second string separately as we checked if both strings
        # are of the same lengths before. If they do so, the second
        # string cannot have all the characters repeated the same number
        # of times as the first one and some other characters as lengths
        # of the strings will not be the same.
        for char in s1:
            if counts[ord(char)] != 0:
                return False
        # Return True if all the counters of characters on the first string
        # are set to zero.
        return True
        
    return are_acronyms


are_acronyms = are_acronyms_init()

###### Kilka testów

In [7]:
print(are_anagrams('aaaa', 'aaaa'))
print(are_anagrams('aaaa', 'aaaaa'))
print(are_anagrams('😂', '😂'))
print(are_anagrams('😂🤣👀🎂✨🎁', '🎂🎁😂👀✨🤣'))
print(are_anagrams('kot', 'kto'))
print(are_anagrams('ola ala ula ela', 'ela ala ola ula'))
print(are_anagrams('ola ala ula ela', 'ela alaola ula'))

True
False
True
True
True
True
False
